In [ ]:
# Instala as bibliotecas necessárias
!pip install openai-whisper openai gTTS

# Instala o ffmpeg (necessário para o Whisper processar áudio)
!sudo apt update && sudo apt install ffmpeg -y

In [ ]:
from IPython.display import display, Javascript
from google.colab import output
import base64

def record_audio(filename='input_audio.wav'):
  js = Javascript('''
    async function recordAudio() {
      const div = document.createElement('div');
      const button = document.createElement('button');
      button.textContent = 'Clique para Gravar (5 seg)';
      button.style.background = 'red';
      button.style.color = 'white';
      button.style.padding = '10px';
      document.body.appendChild(div);
      div.appendChild(button);

      const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
      const recorder = new MediaRecorder(stream);
      const chunks = [];

      return new Promise(resolve => {
        button.onclick = () => {
          recorder.start();
          button.textContent = 'Gravando...';
          setTimeout(() => {
            recorder.stop();
            button.textContent = 'Processando...';
          }, 5000); // Grava por 5 segundos
        };

        recorder.onstop = async () => {
          const blob = new Blob(chunks);
          const reader = new FileReader();
          reader.readAsDataURL(blob);
          reader.onloadend = () => resolve(reader.result);
        };
        recorder.ondataavailable = e => chunks.push(e.data);
      });
    }
  ''')
  display(js)
  data = output.eval_js('recordAudio()')
  binary = base64.b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  print(f"Áudio salvo como {filename}")

# Executa a gravação
record_audio()

In [ ]:
import whisper

# Carrega o modelo (o 'base' é ótimo para velocidade no Colab)
model = whisper.load_model("base")
result = model.transcribe("input_audio.wav")
user_text = result["text"]

print(f"Texto extraído: {user_text}")

In [ ]:
from openai import OpenAI
from google.colab import userdata

client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[{"role": "user", "content": user_text}]
)

chat_response = completion.choices[0].message.content
print(f"Resposta do GPT: {chat_response}")

In [ ]:
from gtts import gTTS
from IPython.display import Audio

# Gera o áudio em português
tts = gTTS(chat_response, lang='pt')
tts.save('response.mp3')

# Exibe o player de áudio com autoplay
display(Audio('response.mp3', autoplay=True))

In [ ]:
from google.colab import userdata
userdata.get('secretName')